In [21]:
import requests
from requests import get
import re
from bs4 import BeautifulSoup as bs
import pandas as pd 
import numpy as np
import random
import string
import csv
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 

In [ ]:
## SCRAPING DATA ##

# Compiling list of directory pages 

url = requests.get('https://www.reed.co.uk/recruiterdirectory')
soup = bs(url.text, 'html.parser')

pages = []

for item in soup.find_all('li', attrs={'class':'page'}):
    pages.append(item.text)
    
final_page = int(items[len(items)-2])

numbers = []
for i in range(1, final_page):
    numbers.append(str(i))
    
page_urls = []

for i in range(0, (final_page-1)):
    base_url = 'http://reed.co.uk/recruiterdirectory?pageno='
    page_url = base_url + numbers[i]
    page_urls.append(page_url)

# Compiling list of employer page urls 

employer_pages = []    

for page in page_urls:
    
    url = requests.get(page)
    soup = bs(url.text, 'html.parser')

    for elm in soup.find_all('div', attrs={'class':'subtitle'}):
        for a_elm in elm.find_all('a'):
            link = a_elm.attrs['href']
            employer_page = 'https://www.reed.co.uk' + link
            employer_pages.append(employer_page)
            print(employer_page)
            
    sleep(randint(0, 5))
    
# Compiling list of employer profiles urls 
    
employer_profiles_raw = []

for i in range(0, len(employer_pages)):
    
    url = requests.get(employer_pages[i])
    soup = bs(url.text, 'html.parser')
    profile = soup.find('a', href=re.compile('employer-profile'))
    employer_profiles_raw.append(profile)
    sleep(random.uniform(5, 15))

employer_profiles = []

for profile_raw in employer_profiles_raw:
    if profile_raw != None:
        profile_string = str(profile_raw)
        search_str = re.search(''(.+?)'', profile_string)
        link_suf = search_str.group(1)
        full_link = 'https://www.reed.co.uk' + link_suf
        employer_profiles.append(full_link)
        print(full_link)
    else:
        employer_profiles.append('No profile')
        print('No profile')
        
# Exporting employer pages and profiles to df 

pages_profiles = pd.DataFrame(list(zip(employer_pages, employer_profiles)), columns=['employer_page', 'employer_profile'])
pages_profiles.to_csv('/Users/elenaleggett/Documents/Masters/Dissertation/gender_pay_gap/data_scripts/data/pages_profiles.csv')

# Removing companies with no profiles

profile_pages = []

for profile in employer_profiles:
    if profile != 'No profile':
        profile_pages.append(profile)

jobs_pages = []

for i in range(0, len(employer_pages)):
    if employer_profiles[i] != 'No profile':
        jobs_pages.append(employer_pages[i]) 
        
# Compiling profile page variables 

names = []
sizes = []
sectors = []
what_we_dos = []
what_youll_gets = []
who_youll_work_withs = []

for i in range(0, len(profile_pages)):
    
    url = requests.get(profile_pages[i])
    soup = bs(url.text, 'html.parser')
    
    # Name
    
    title_raw = soup.find('span', {'class':'title'})
    title = title_raw.string
    name = title[11:]
    names.append(name.lower())
    
    # Size 
    
    size_raw = soup.find('span', {'class':'information__description'})
    size = size_raw.string
    sizes.append(size)
    
    # Sector
    
    sector_raw = size_raw.find_next('span', {'class':'information__description'})
    sector = sector_raw.string
    sectors.append(sector)
    
    # What we do 
    
    do_heading = soup.find(text = re.compile('What we do'), attrs = {'class':'heading'})

    if do_heading != None:
        do_text = do_heading.find_next_sibling('div')
        do_str = str(do_text)
        a = re.compile('<.*?>')
        do_str = a.sub(' ', do_str)
        do_str = do_str.replace('\n', '')
        do_str = do_str.replace('\xa0', '')
    else: 
        do_str = 'Empty'
        
    what_we_dos.append(do_str)
        
    # What you'll get 
    
    get_heading = soup.find(text = re.compile('What you'll get'), attrs = {'class':'heading'})

    if get_heading != None:
        get_text = get_heading.find_next_sibling('ul')
        get_str = str(get_text.get_text())
        get_str = get_str.replace('\n\n\n\n', '; ')
        get_str = get_str.replace('\n', '')

    else:
        get_str = 'Empty'
    
    what_youll_gets.append(get_str)
    
    # Who you'll work with
    
    who_heading = soup.find(text = re.compile('Who you'll work with'), attrs = {'class':'heading'})

    if who_heading != None:

        who_text = who_heading.find_next_sibling('div')
        who_str = str(who_text)
        a = re.compile('<.*?>')
        who_str = a.sub(' ', who_str)    
        who_str = who_str.replace('\n', '')
        who_str = who_str.replace('\xa0', '')

    else:
        who_str = 'Empty'
        
    who_youll_work_withs.append(who_str)
    
    sleep(random.uniform(5, 15))
    
# Jobs page variables 

ft_jobs = []
pt_jobs = []

for i in range(0, len(jobs_pages)):

    url = requests.get(jobs_pages[i])
    soup = bs(url.text, 'html.parser')
    
    # FT jobs
    
    ft_raw = soup.find('li', {'data-jobtype':'FullTime'})
    ft_count = ft_raw.find('span', {'class':'count'})
    ft_text = ft_count.get_text()
    ft_text = ft_text.replace('(', '')
    ft_text = ft_text.replace(')', '')
    ft_text = ft_text.replace(',', '')
    ft_jobs.append(ft_text)
    
    # PT jobs
    
    pt_raw = soup.find('li', {'data-jobtype':'PartTime'})
    pt_count = pt_raw.find('span', {'class':'count'})
    pt_text = pt_count.get_text()
    pt_text = pt_text.replace('(', '')
    pt_text = pt_text.replace(')', '')
    pt_text = pt_text.replace(',', '')
    pt_jobs.append(pt_text)
    
    sleep(random.uniform(5, 15))
    

## PROCESSING DATA ##
    
    
# Processing text columns 

what_we_dos_edited = []

for element in what_we_dos:
    element_lower = element.lower()
    what_we_dos_edited.append(element_lower)
    
what_we_dos_final = []

for element in what_we_dos_edited:
    regex = re.compile('[^a-zA-Z ]')
    element = regex.sub('', element)
    element = ' '.join([word for word in element.split() if word not in stopwords.words()])
    what_we_dos_final.append(element)
    
what_youll_gets_edited = []

for element in what_youll_gets:
    element_lower = element.lower()
    what_youll_gets_edited.append(element_lower)
    
what_youll_gets_final = []

for element in what_youll_gets_edited:
    regex = re.compile('[^a-zA-Z ]')
    element = regex.sub('', element)
    element = ' '.join([word for word in element.split() if word not in stopwords.words()])
    what_youll_gets_final.append(element)
    
who_youll_work_withs_edited = []

for element in who_youll_work_withs:
    element_lower = element.lower()
    who_youll_work_withs_edited.append(element_lower)
    
who_youll_work_withs_final = []

for element in who_youll_work_withs_edited:
    regex = re.compile('[^a-zA-Z ]')
    element = regex.sub('', element)
    element = ' '.join([word for word in element.split() if word not in stopwords.words()])
    who_youll_work_withs_final.append(element)
    
agentic = ['control ', 'controls ', 'controlling ', 'controller ', 'initiative ', 'initiatives ', 'motivation ', 'motivations ', 'motivated ', 'pressure ', 'pressured ', 'pressures ', 'productive ', 'productivity ', 'responsible ', 'responsibilities ', 'power ', 'powerful ', 'competitive ', 'driven ', 'drive ']
communal = ['punctual', 'punctuality', 'honest', 'honesty', 'attentive', 'attentiveness', 'teamwork', 'helpful', 'helpfulness', 'courteous', 'courtesy', 'courteousness']
inclusive = ['diversity ', 'diverse ', 'inclusion ', 'inclusive ', 'family friendly ', 'parental ', 'flexible ', 'flexibility ', 'collective ', 'collectivity ', 'equal ', 'equality ', 'share ', 'sharing ']

agentic_counts_wwd = []

agentic_count = 0

for description in what_we_dos_final:
    agentic_count = 0
    for word in agentic:
        if word in description:
            agentic_count = agentic_count + 1
        else:
            agentic_count = agentic_count + 0 
    agentic_counts_wwd.append(agentic_count)
    
communal_counts_wwd = []

communal_count = 0

for description in what_we_dos_final:
    communal_count = 0
    for word in communal:
        if word in description:
            communal_count = communal_count + 1
        else:
            communal_count = communal_count + 0 
    communal_counts_wwd.append(communal_count)
    
inclusive_counts_wwd = []

inclusive_count = 0

for description in what_we_dos_final:
    inclusive_count = 0
    for word in inclusive:
        if word in description:
            inclusive_count = inclusive_count + 1
        else:
            inclusive_count = inclusive_count + 0 
    inclusive_counts_wwd.append(inclusive_count)
    
agentic_counts_wyg = []

agentic_count = 0

for description in what_youll_gets_final:
    agentic_count = 0
    for word in agentic:
        if word in description:
            agentic_count = agentic_count + 1
        else:
            agentic_count = agentic_count + 0 
    agentic_counts_wyg.append(agentic_count)
    
communal_counts_wyg = []

communal_count = 0

for description in what_youll_gets_final:
    communal_count = 0
    for word in communal:
        if word in description:
            communal_count = communal_count + 1
        else:
            communal_count = communal_count + 0 
    communal_counts_wyg.append(communal_count)
    
inclusive_counts_wyg = []

inclusive_count = 0

for description in what_youll_gets_final:
    inclusive_count = 0
    for word in inclusive:
        if word in description:
            inclusive_count = inclusive_count + 1
        else:
            inclusive_count = inclusive_count + 0 
    inclusive_counts_wyg.append(inclusive_count)
    
agentic_counts_wyww = []

agentic_count = 0

for description in who_youll_work_withs_final:
    agentic_count = 0
    for word in agentic:
        if word in description:
            agentic_count = agentic_count + 1
        else:
            agentic_count = agentic_count + 0 
    agentic_counts_wyww.append(agentic_count)
    
communal_counts_wyww = []

communal_count = 0

for description in who_youll_work_withs_final:
    communal_count = 0
    for word in communal:
        if word in description:
            communal_count = communal_count + 1
        else:
            communal_count = communal_count + 0 
    communal_counts_wyww.append(communal_count)
    
inclusive_counts_wyww = []

inclusive_count = 0

for description in who_youll_work_withs_final:
    inclusive_count = 0
    for word in inclusive:
        if word in description:
            inclusive_count = inclusive_count + 1
        else:
            inclusive_count = inclusive_count + 0 
    inclusive_counts_wyww.append(inclusive_count)
    
# % part time 

prop_pt = []

ft_nums = []
pt_nums = []

for i in range(0, len(ft_jobs)):
    ft_num = int(ft_jobs[i])
    ft_nums.append(ft_num)
    pt_num = int(pt_jobs[i])
    pt_nums.append(pt_num)
               
for i in range(0, len(ft_jobs)):
    if pt_nums[i] != 0:
        if ft_nums[i] != 0:
            prop_pt_element = pt_nums[i] / ft_nums[i]
            prop_pt_element = prop_pt_element * 100
            prop_pt_element = round(prop_pt_element, 3)
            prop_pt.append(prop_pt_element)
        else:
            prop_pt.append(100)
    else:
        prop_pt.append(0)
        

# Converting categorical sizes column into numerical

sizes_modified = []

for i in range(0, len(sizes)):
    if sizes[i] == '1–4 employees':
        sizes_modified.append(1)
    if sizes[i] == '5–9 employees':
        sizes_modified.append(5)
    if sizes[i] == '10–19 employees':
        sizes_modified.append(10)
    if sizes[i] == '20–49 employees':
        sizes_modified.append(20)
    if sizes[i] == '50–99 employees':
        sizes_modified.append(50)
    if sizes[i] == '100–249 employees':
        sizes_modified.append(100)
    if sizes[i] == '250–499 employees':
        sizes_modified.append(250)
    if sizes[i] == '500–999 employees':
        sizes_modified.append(500)
    if sizes[i] == '1,000–2,499 employees':
        sizes_modified.append(1000)
    if sizes[i] == '2,500–4,999 employees':
        sizes_modified.append(2500)
    if sizes[i] == '5,000+ employees':
        sizes_modified.append(5000)
        
# Cleaning name for matching 

names_lower = []
for name in names:
    name_lower = name.lower()
    names_lower.append(name_lower)
    
names_cleaned_0 = []
for name in names_lower:
    if ' ltd' in name:
        newname = name.replace(' ltd', ' limited')
        names_cleaned_0.append(newname)
    else:
        names_cleaned_0.append(name)

names_cleaned = []
for name in names_cleaned_0:
    if ' limited' in name:
        newname = name.replace(' limited', '')
        names_cleaned.append(newname)
    else:
        names_cleaned.append(name)
        
# Forming Reed data frame and exporting to csv 

columns = ['name', 'size', 'sector', 'prop_pt', 'what_we_do_agentic', 'what_we_do_communal', 'what_we_do_inclusive', 'what_youll_get_agentic', 'what_youll_get_communal', 'what_we_do_inclusive', 'who_youll_work_with_agentic', 'who_youll_work_with_communal', 'who_youll_work_with_inclusive']
reed_data = pd.DataFrame(list(zip(names_cleaned, sizes_modified, prop_pt, agentic_counts_wwd, communal_counts_wwd, inclusive_counts_wwd, agentic_counts_wyg, communal_counts_wyg, inclusive_counts_wyg, agentic_counts_wyww, communal_counts_wyww, inclusive_counts_wyww)), columns = columns)
reed_data.to_csv('reed.data.csv')
                         
# Editing name in UK gpg dataset 
                         
uk_gpg_all = pd.read_csv('https://gender-pay-gap.service.gov.uk/viewing/download-data/2022')    
                         
gpg_names = list(uk_gpg_data['EmployerName'])
gpg_names_lower = []
for name in gpg_names:
    name_lower = name.lower()
    gpg_names_lower.append(name_lower)
    
gpg_names_cleaned_0 = []
for name in gpg_names_lower:
    if ' ltd' in name:
        newname = name.replace(' ltd', ' limited')
        gpg_names_cleaned_0.append(newname)
    else:
        gpg_names_cleaned_0.append(name)

gpg_names_cleaned = []
for name in gpg_names_cleaned_0:
    if ' limited' in name:
        newname = name.replace(' limited', '')
        gpg_names_cleaned.append(newname)
    else:
        gpg_names_cleaned.append(name)
                                           
# Forming gpg data frame and exporting to csv 
                         
gpg = uk_gpg_all['DiffMeanHourlyPercent']

columns = ['name', 'gpg'] 
uk_gpg = pd.DataFrame(list(zip(gpg_names_cleaned, gpg)), columns = columns)
mean_gpg = uk_gpg['DiffMeanHourlyPercent'].mean()
uk_gpg['gpg_above_average'] = np.where(uk_gpg_all['DiffMeanHourlyPercent'] > mean_gpg, 'Above', 'Below')
uk_gpg = uk_gpg.drop(columns=['gpg'])                                            
                                           
gpg_data.to_csv('gpg_data')
